In [1]:
import requests

# Datos de la API
base_url = "http://development.localhost:8000"
api_key = "8d2f08387ea6aca"
api_secret = "9b887566130a4cd"

# Headers para autenticación
headers = {
    "Authorization": f"token {api_key}:{api_secret}"
}

# Endpoint que deseas consumir (ejemplo: listar documentos de la Doctype 'Item')
endpoint = "/api/method/frappe.auth.get_logged_user"

# Hacer la petición GET
response = requests.get(f"{base_url}{endpoint}", headers=headers)

# Ver el resultado
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code} - {response.text}")

{'message': 'Administrator'}


In [2]:
def link_transfer(name, doctype, links: dict):
    data = {
        f"customer_primary_{doctype.lower()}": name,  # ID del contacto como primario
    }

    for link in links:
        endpoint = f"/api/resource/{link['link_doctype']}/{link['link_name']}"
        response = requests.put(f"{base_url}{endpoint}", json=data, headers=headers)
    
    endpoint_link = f"/api/resource/{doctype}/{name}"
    data = {"links": links}
    response_link = requests.put(f"{base_url}{endpoint_link}", json=data, headers=headers)
    if response_link.status_code == 200 and response.status_code == 200:
        print(f"{link['link_name']} actualizado exitosamente")

In [3]:
def create(doctype: str, data: dict, links: dict = None) -> str:
    endpoint = f'/api/resource/{doctype}'

    response = requests.post(f"{base_url}{endpoint}", json=data, headers=headers)

    name = None
    
    if links: data['links'] = links

    if response.status_code == 200:
        print(f"{doctype} creado exitosamente:")
        resp = response.json()
        print(resp)
        name = resp['data']['name']
        if links: link_transfer(name, doctype, data['links'])
    else:
        print(f"Error al crear el {doctype}: {response.status_code} - {response.text}")
        
    
    return name

In [4]:
from api.sync import Sync

client = Sync('http://192.168.99.102/vtigercrm_2022', 'superadmin', 'MFaeyxCMTmRrUZiE')

service_id = "24x3"
product_id = "14x11"
assigned_user_id = "19x1"

19x113


In [5]:
module = "SalesOrder"
condition = {
    "cf_2059" :"2025-01-01"
}

module_object = client.search(module, condition)

In [6]:
def create_dependent_data(salesorder_data):
    mapping_cf = [
        {
            "type": "spouse",
            "firstname": "cf_2347",
            "middlename": "cf_2349",
            "lastname": "cf_2351",
            "ssn": "cf_2357",
            "dob": "cf_2355",
            "gender": "cf_2359",
            "apply": "cf_2389",
            "smoke": "cf_2709",
            "pregnant": "cf_2383",
            "relationship": "",
            "document": "cf_2603",
        },
        {
            "type": "dependent_1",
            "firstname": "cf_2405",
            "middlename": "cf_2407",
            "lastname": "cf_2409",
            "ssn": "cf_2417",
            "dob": "cf_2413",
            "gender": "cf_2411",
            "apply": "cf_2401",
            "smoke": "cf_2711",
            "pregnant": "cf_2423",
            "relationship": "cf_2431",
            "document": "cf_2605"
        },
        {
            "type": "dependent_2",
            "firstname": "cf_2443",
            "middlename": "cf_2445",
            "lastname": "cf_2447",
            "ssn": "cf_2455",
            "dob": "cf_2451",
            "gender": "cf_2449",
            "apply": "cf_2439",
            "smoke": "cf_2713",
            "pregnant": "cf_2459",
            "relationship": "cf_2465",
            "document": "cf_2607"
        },
        {
            "type": "dependent_3",
            "firstname": "cf_2479",
            "middlename": "cf_2481",
            "lastname": "cf_2483",
            "ssn": "cf_2491",
            "dob": "cf_2487",
            "gender": "cf_2485",
            "apply": "cf_2475",
            "smoke": "cf_2715",
            "pregnant": "cf_2495",
            "relationship": "cf_2501",
            "document": "cf_2609"
        },
        {
            "type": "dependent_4",
            "firstname": "cf_2515",
            "middlename": "cf_2517",
            "lastname": "cf_2519",
            "ssn": "cf_2527",
            "dob": "cf_2523",
            "gender": "cf_2521",
            "apply": "cf_2511",
            "smoke": "cf_2717",
            "pregnant": "cf_2531",
            "relationship": "cf_2537",
            "document": "cf_2639"
        },
        {
            "type": "dependent_5",
            "firstname": "cf_2645",
            "middlename": "cf_2647",
            "lastname": "cf_2649",
            "ssn": "cf_2687",
            "dob": "cf_2681",
            "gender": "cf_2679",
            "apply": "cf_2615",
            "smoke": "cf_2719",
            "pregnant": "cf_2685",
            "relationship": "cf_2699",
            "document": "cf_2683"
        }
    ]
    
    owner_data = []
    
    for dependent_cf in mapping_cf:
        if salesorder_data[dependent_cf['apply']] != '':
            owner_data.append(
                {
                    "first_name": salesorder_data[dependent_cf['firstname']],
                    "middle_name": salesorder_data[dependent_cf['middlename']],
                    "last_name": salesorder_data[dependent_cf['lastname']],
                },
            )
    
    return owner_data

In [7]:
def create_dependent(salesorder_data):
    dependent_data = create_dependent_data(salesorder_data)
    dependent_name = []
    for data in dependent_data:
        dependent_name.append(create('Contact', data))
    return dependent_name

In [8]:
import json
def get_id(firstname, middlename, lastname):
    endpoint = f'/api/resource/Customer'
    
    filters = [
        ["name", "=", f"{firstname} {middlename} {lastname}"]
    ]

    response = requests.get(f"{base_url}{endpoint}", params={'filters': json.dumps(filters)}, headers=headers)

    if response.status_code == 200:
        resp = response.json()
        if len(resp['data']) > 0:
            return False
        else:
            print(f"No existe nadie llamado: {firstname} {middlename} {lastname}")
            return True 
    else:
        print(f"Error al filtrar: {filters}")
        return False

In [9]:
get_id("MARIA", "ANTONIA", "PENA DUARTE")

False

In [18]:
def create_owner_data(salesorder_data):
    contact = client.get_contact_from_id(salesorder_data['contact_id'])
    
    owner_data = []
    
    if get_id(contact['firstname'], contact['middlename'], contact['lastname']):
        owner_data = {
            "contact_data": {
                "first_name": contact['firstname'],
                "middle_name": contact['middlename'],
                "last_name": contact['lastname'],
                "custom_day_of_birth": contact['dob'],
                "custom_ssn": ''.join([char for char in contact['ssn'] if char.isdigit()]),
                "gender": str(contact['gender']).lower().capitalize(),
                "email_ids": [
                    {
                        "email_id": contact['email'],  
                        "is_primary": 1,
                    }
                ],
                "phone_nos": [
                    {
                        "phone": contact['phone_1'],
                        "is_primary": 1,
                    }
                ]
            },
            "customer_data": {
                "customer_name": f"{contact['firstname']} {contact['middlename']} {contact['lastname']}",
                "customer_type": "Individual",
                "territory": "All Territories"
            },
            "address_data": {
                "address_title": f"{contact['firstname']} {contact['middlename']} {contact['lastname']}",
                "address_line1": salesorder_data['cf_2737'],
                "city": salesorder_data['cf_2739'],
                "state": salesorder_data['cf_2765'],
                "pincode": salesorder_data['cf_2743'],
                "country": "United States",
            }
        }

    return owner_data

In [11]:
def create_owner(salesorder_data):
    customer_name = None
    
    if owner_data := create_owner_data(salesorder_data):
        customer_name = create('Customer', owner_data['customer_data'])
        customer_links = [
            {
                "link_doctype": "Customer",
                "link_name": customer_name
            }
        ]
        create('Address', owner_data['address_data'], customer_links)
        create('Contact', owner_data['contact_data'], customer_links)
        
    return customer_name

In [12]:
def create_salesorder(customer_name: str):
    endpoint = f'/api/resource/Sales Order'
    
    data = {
        "customer": customer_name,  # Reemplaza por el ID del cliente
        "delivery_date": "2025-01-01",  # Fecha de entrega
        "items": [
            {
                "item_code": "BS",  # Reemplaza por el código del artículo
                "qty": 1,
                "rate": 100.0  # Precio del artículo
            }
        ],
        "currency": "USD",  # Moneda (puede cambiarse según la configuración)
        "selling_price_list": "Standard Selling",  # Lista de precios
    }

    response = requests.post(f"{base_url}{endpoint}", json=data, headers=headers)

    if response.status_code == 200:
        print("Sales Order creada exitosamente:", response.json())
    else:
        print("Error al crear la Sales Order:", response.status_code, response.text)

In [16]:
salesorder_data = module_object[24]

In [17]:
owner_name = create_owner(salesorder_data)
salesorder_name = create_salesorder(owner_name)
#dependent_name = create_dependent(salesorder_data)


No existe nadie llamado: DANIELLA PAOLA MONCADA
Customer creado exitosamente:
{'data': {'name': 'DANIELLA PAOLA MONCADA', 'owner': 'Administrator', 'creation': '2024-10-08 11:39:15.632053', 'modified': '2024-10-08 11:39:15.632053', 'modified_by': 'Administrator', 'docstatus': 0, 'idx': 0, 'naming_series': 'CUST-.YYYY.-', 'customer_name': 'DANIELLA PAOLA MONCADA', 'customer_type': 'Individual', 'territory': 'All Territories', 'is_internal_customer': 0, 'language': 'en-US', 'default_commission_rate': 0.0, 'so_required': 0, 'dn_required': 0, 'exempt_from_sales_tax': 0, 'is_frozen': 0, 'disabled': 0, 'doctype': 'Customer', 'sales_team': [], 'credit_limits': [], 'portal_users': [], 'companies': [], 'accounts': []}}
Address creado exitosamente:
{'data': {'name': 'DANIELLA PAOLA MONCADA-Billing', 'owner': 'Administrator', 'creation': '2024-10-08 11:39:15.726999', 'modified': '2024-10-08 11:39:15.726999', 'modified_by': 'Administrator', 'docstatus': 0, 'idx': 0, 'address_title': 'DANIELLA PAOL

In [15]:
owner_name

'GABY  MERINO NATTERI'